In [1]:
import pandas as pd
import folium 
import numpy as np
from datetime import datetime as dt
from connect_to_mysql import Database
import networkx as nx
import collections 
pd.set_option("display.max_columns", None)

In [ ]:
#repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")
#df = pd.read_csv(f"{repo}/data/cleaning/test_1651834681.csv")
df = pd.read_csv("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/mtd_combined.csv")
df = df[(df["state"] == "completed")]

In [ ]:
Database.query_dataframe('rides', 'Show tables')

In [ ]:
#df_edges = Database.query_dataframe('stopconnection', "Select * From stopconnection")
df_edges = pd.read_excel("/Users/ericchittka/Documents/Daten/MoDstops+Preismodell.xlsx")

In [ ]:
df_edges.dtypes

### Number of Drives 

In [ ]:
df.columns

In [ ]:
pd.to_datetime(df["scheduled_to"])[0].month

In [ ]:
frequencies = pd.DataFrame(df[(pd.to_datetime(df["scheduled_to"]).dt.month == 2) & (pd.to_datetime(df["scheduled_to"]).dt.year == 2022) ].groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()

frequencies

In [ ]:
frequencies["pickup_address"] = frequencies["pickup_address"].astype(str)
frequencies["dropoff_address"] = frequencies["dropoff_address"].astype(str)

In [ ]:
#df_edges = pd.read_excel(
#        f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="Liste 2022"
#)

In [ ]:
df_edges.columns

In [ ]:
df_edges = df_edges[["id_start", "id_end"]]

In [ ]:
df_edges["id_start"] = df_edges["id_start"].astype(str)
df_edges["id_end"] = df_edges["id_end"].astype(str)

In [ ]:
edges = pd.merge(df_edges, frequencies,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "number_of_drives"]]
edges = edges[edges["number_of_drives"].isna() == False]
edges["number_of_drives"] = 28  / edges["number_of_drives"] 

In [ ]:
edges

In [ ]:
edges["number_of_drives"].fillna(0, inplace=True)

In [ ]:
#G = nx.from_pandas_edgelist(
#    edges,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "number_of_drives"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G)
#plt.show()

In [ ]:
#print(nx.shortest_path(G, source="10002", target="4007", weight = "number_of_drives"))
#print(nx.shortest_path_length(G, source="10002", target="4007", weight="number_of_drives", method='dijkstra'))

### Average Ride Time per drive  

In [ ]:
#df
#df.dtypes['ride_time']

In [ ]:
average = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean())
average.rename(columns = {"ride_time":"avg_ride_time"}, inplace = True)
average

In [ ]:
average["pickup_address"] = average["pickup_address"].astype(str)
average["dropoff_address"] = average["dropoff_address"].astype(str)

In [ ]:
edges2 = pd.merge(df_edges, average,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "avg_ride_time"]]
edges2 = edges2[edges2["avg_ride_time"].isna() == False]
edges2

In [ ]:
#G1 = nx.from_pandas_edgelist(
#    edges2,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "avg_ride_time"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G1)
#plt.show()

In [ ]:
#print(nx.shortest_path(G1, source="1001", target="9015", weight = "avg_ride_time"))
#print(nx.shortest_path_length(G1, source="1001", target="9015", weight="avg_ride_time", method='dijkstra'))

### Original Function

In [ ]:
def get_shortest_ride(startpoint, endpoint, start_date, end_date):
    start_date = dt.strptime(start_date, "%m/%d/%Y")
    end_date = dt.strptime(end_date, "%m/%d/%Y")
    days = (end_date - start_date).days + 1
    df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
    df = df[(df["scheduled_to"] > start_date) & (df["scheduled_to"] < end_date)]
    
    drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
    
    drives["waiting_time"] = days / drives["number_of_drives"]

    drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
    
    drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )

    path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
    shortest_time = nx.shortest_path_length(G, source=startpoint, target=endpoint, weight="avg_time_to_destination", method='dijkstra') 

    return (path, shortest_time)

### Set start and end date 

In [2]:
df = pd.read_csv("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/cleaning/test_1656252990.csv")
df = df[(df["state"] == "completed")]

In [3]:
#df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
startdate = df.scheduled_to.min()
startdate = dt.strptime(startdate, '%Y-%m-%d %H:%M:%S')
startdate = startdate.strftime('%m/%d/%Y')
start_date = dt.strptime(startdate, "%m/%d/%Y")
enddate = df.scheduled_to.max()
enddate = dt.strptime(enddate, '%Y-%m-%d %H:%M:%S')
enddate = enddate.strftime('%m/%d/%Y')
end_date = dt.strptime(enddate, "%m/%d/%Y")

In [4]:
#df['pickup_address'] = df['pickup_address'].map(str)
#df['dropoff_address'] = df['dropoff_address'].map(str)

### Drive DF

In [5]:
days = (end_date - start_date).days + 1
drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
drives["waiting_time"] = days / drives["number_of_drives"]
drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

In [6]:
drives

,pickup_address,dropoff_address,number_of_drives,waiting_time,avg_ride_time,avg_time_to_destination
0,1001,1004,5,54.800000,0.001738,54.801738
1,1001,1005,18,15.222222,0.001552,15.223774
2,1001,1008,3,91.333333,0.003538,91.336871
3,1001,1012,1,274.000000,0.007627,274.007627
4,1001,1013,1,274.000000,0.005856,274.005856
...,...,...,...,...,...,...
2271,18005,7001,4,68.500000,0.006704,68.506704
2272,18005,9001,1,274.000000,0.010185,274.010185
2273,18005,9005,6,45.666667,0.009236,45.675903
2274,18005,9006,1,274.000000,0.009155,274.009155


### Graph

In [7]:
G = nx.from_pandas_edgelist(
    drives,
    source = "pickup_address",
    target = "dropoff_address",
    edge_attr= "avg_time_to_destination"
)

In [8]:
def calculate_graph(start_date, end_date):
  
    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )
    return G

In [9]:
def get_shortest_ride(startpoint, endpoint, G):

    if startpoint not in G:
        return 'Not in G'
    elif endpoint not in G: 
        return 'Not in G'
    else:
        path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
        return path

In [10]:
# df_test = pd.DataFrame({'Start':['1001', '1002', '1931391'], 'End': ['10004', '1008', '32432432']})
# df_test["Start"] = df_test["Start"].astype(str).astype(int)
# df_test["End"] = df_test["End"].astype(str).astype(int)
# df_test['HotSpots'] = df_test.apply(lambda x: get_shortest_ride(x.Start, x.End, G), axis=1)
# df_test

In [11]:
df_edges = pd.read_excel("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/other/MoDstops+Preismodell.xlsx", sheet_name= 'Liste 2022')

In [12]:
df_edges.rename(columns= {'Start #':'start_id', 'Ende #':'end_id'}, inplace = True)
df_edges['Spots'] = df_edges.apply(lambda x: get_shortest_ride(x.start_id, x.end_id, G), axis=1)

In [13]:
df_edges = df_edges[df_edges.Spots != 'Not in G']

In [14]:
hotSpots_list = list(df_edges['Spots'])

In [15]:
flat_hotSpots_list = [x for xs in hotSpots_list for x in xs]

In [16]:
counter=collections.Counter(flat_hotSpots_list)
hotSpots=pd.DataFrame(counter.most_common(15), columns=['id', 'frequency'])

### Adding Drone flights 

In [17]:
drone_flights = df_edges.iloc[:,:6]
drone_flights.drop(['Start Name', 'Ende Name', 'Route [m]'], axis=1, inplace=True)
drone_flights.rename(columns = {'Luftlinie [m]':'Luftlinie'}, inplace = True)
drone_flights = drone_flights[drone_flights.Luftlinie <= 500]
drone_flights 

,start_id,end_id,Luftlinie
0,1001,1002,285
8,1001,1010,314
22,1001,3001,337
29,1001,3009,243
32,1001,3012,386
...,...,...,...
39382,15015,15001,434
39384,15015,15003,380
39385,15015,15004,421
39386,15015,15005,223


In [18]:
# Annahme: Drohne fliegt 30 km/h --> 12 Sekunden pro 100 Meter und dann in Tage umrechnen  
drone_flights['avg_time_to_destination'] = (drone_flights.Luftlinie * 0.12) /60 /60 /24

In [19]:
drives_w_flights = pd.merge(drives, drone_flights, left_on=["pickup_address", "dropoff_address"], right_on=["start_id", "end_id"], how="left")

In [20]:
drives_w_flights['avg_time_to_destination'] = np.where((drives_w_flights['avg_time_to_destination_x'] > drives_w_flights['avg_time_to_destination_y']),
                                                drives_w_flights.avg_time_to_destination_y,
                                                drives_w_flights.avg_time_to_destination_x)

In [21]:
G1 = nx.from_pandas_edgelist(
    drives_w_flights,
    source = "pickup_address",
    target = "dropoff_address",
    edge_attr= "avg_time_to_destination"
)

In [22]:
df_edges1 = pd.read_excel("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/other/MoDstops+Preismodell.xlsx", sheet_name= 'Liste 2022')

In [23]:
df_edges1.rename(columns= {'Start #':'start_id', 'Ende #':'end_id'}, inplace = True)
df_edges1['Spots'] = df_edges1.apply(lambda x: get_shortest_ride(x.start_id, x.end_id, G1), axis=1)

### Map HotSpots

In [24]:
df_spots = pd.read_excel("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/other/MoDstops+Preismodell.xlsx")

In [25]:
df_hotSpots = pd.merge(df_spots, hotSpots, left_on=['MoDStop Id'], right_on=['id'], how="left")
df_hotSpots

,MoDStop Id,MoDStop Name,MoDStop Lat,MoDStop Long,MoDStop Adresse,id,frequency
0,1001,Mandelgasse,49.351780,8.129000,"Seilerbahn 1, 67433 Neustadt",NaN,NaN
1,1002,Hauptfeuerwache,49.353733,8.131552,"Lindenstraße 11, 67433 Neustadt",1002.0,4170.0
2,1003,Kindergarten St. Marien,49.355745,8.134341,"Sauterstraße 12, 67433 Neustadt",NaN,NaN
3,1004,Strohmarkt,49.355520,8.136674,"Strohmarkt, 67433 Neustadt",1004.0,1410.0
4,1005,Rathaus,49.354022,8.135143,"Rathausstraße 1, 67433 Neustadt",1005.0,10060.0
...,...,...,...,...,...,...,...
196,15015,Am Krautgarten,49.366103,8.181919,"Am Krautgarten 4, 67435 Neustadt",NaN,NaN
197,16009,In den Obergärten,49.335468,8.203290,"Flugplatzstraße 11, 67435 Neustadt",NaN,NaN
198,16027,Altes Schulhaus Lachen,49.322493,8.196059,"Theodor-Heuss-Straße 29, 67435 Neustadt",NaN,NaN
199,17007,Duttweiler Dorfplatz,49.299327,8.211596,"Dudostraße 62, 67435 Neustadt",NaN,NaN


In [26]:
df_hotSpots['HotSpot'] = np.where(df_hotSpots.frequency.notnull(), 'Yes', 'No')

In [27]:
def select_marker_color(row):
    if row['HotSpot'] == 'Yes':
        return 'red'
    return 'blue'

In [28]:
df_hotSpots['color'] = df_hotSpots.apply(select_marker_color, axis=1)

In [51]:
hotSpot_map = folium.Map(
    location = [49.340510, 8.133150], 
    zoom_start = 12
)
for index, row in df_hotSpots.iterrows():
    folium.Marker(
        location=[row['MoDStop Lat'], row['MoDStop Long']],
        popup=row['frequency'],
        tooltip=row['MoDStop Name'],
        icon=folium.Icon(color=row['color'],prefix='fa', icon='circle')
    ).add_to(hotSpot_map)

In [52]:
hotSpot_map.save("hotSpot_map.html")

In [53]:
hotSpot_map